In [12]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

examples = [
    {
        "question": "영화 탑건, 3개의 이모지로 요약해주세요.",
        "answer": "🛩️👨‍✈️🔥"
    },
    {
        "question": "영화 대부, 3개의 이모지로 요약해주세요.",
        "answer": "👨‍👨‍👦🔫🍝"
    }
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}\n")
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Human: 영화 {movie}, 3개의 이모지로 요약해주세요.",
    input_variables=["movie"]
)

chain = prompt | chat 

chain.invoke({"movie": "어벤져스: 엔드게임"})

AI: 🦸‍♂️💥⚔️

AIMessageChunk(content='AI: 🦸\u200d♂️💥⚔️')

In [16]:

memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_token_limit=120,
    return_messages=True,
)


new_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 매우 재치있는 영화 리뷰어입니다. 당신은 영화를 3개의 이모지로 요약할 수 있습니다."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | new_prompt | chat


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

invoke_chain("다크나이트, 3개의 이모지로 요약해주세요.")
invoke_chain("에일리언, 3개의 이모지로 요약해주세요.")
invoke_chain("스타워즈, 3개의 이모지로 요약해주세요.")

🦇🌃🤯content='🦇🌃🤯'
👽🚀😱content='👽🚀😱'
🌌⚔️🤖The human asks the AI to summarize "The Dark Knight" in 3 emojis.content='🌌⚔️🤖'


In [17]:
invoke_chain("처음에 질문한 영화는 무엇인가요")

처음에 질문한 영화는 "에일리언"입니다.The human asks the AI to summarize "The Dark Knight" in 3 emojis. The AI responds with 🦇🌃🤯. The human then asks the AI to summarize "Alien" in 3 emojis.content='처음에 질문한 영화는 "에일리언"입니다.'
